# Bokeh Tutorial
## 4 - Final Project
### Cristobal Donoso 
##### may 11, 2019
<font size="1">*This tutorial is based on [the official documentation](https://bokeh.pydata.org/en/latest/docs/user_guide.html
)*</font>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
tf.__version__

'2.0.0-beta1'

In [5]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

In [7]:
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
train_images, test_images = train_images / 255.0, test_images / 255.0

### Defining the model

In [8]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [9]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=1)

W0708 21:06:09.040155 140715783341888 deprecation.py:323] From /opt/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples
60000/60000 [==============================] - 25s 409us/sample - loss: 0.1439 - accuracy: 0.9557


In [11]:
y_pred = model.predict_classes(test_images)

### Building an interactive confusion matrix

In [12]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [13]:
from bokeh.models import LinearColorMapper, FixedTicker,TapTool, ColorBar, ColumnDataSource, CustomJS
from bokeh.plotting import figure
from sklearn.metrics import confusion_matrix
import numpy as np

In [14]:
cm = confusion_matrix(test_labels, y_pred)
cm = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], 2)
cm

array([[0.98, 0.  , 0.  , 0.  , 0.01, 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.01, 0.  , 0.98, 0.01, 0.  , 0.01, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.01, 0.01, 0.  , 0.  , 0.  , 0.97, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.99, 0.  ],
       [0.  , 0.01, 0.  , 0.  , 0.02, 0.  , 0.  , 0.01, 0.01, 0.95]])

In [15]:
accuracies = cm.flatten() # accuracies for each combination

In [16]:
n_classes = 10
y_true_lab = np.array([np.ones(n_classes)*i for i in range(0, n_classes)]).flatten()
y_pred_lab = np.array([np.arange(0, n_classes) for i in range(0, n_classes)]).flatten()

### ColumnDataSource 

In [49]:
data = {'y_true': y_true_lab, 'y_pred': y_pred_lab, 'acc': accuracies}
source = ColumnDataSource(data)

### Defining color [palette](https://bokeh.pydata.org/en/latest/docs/reference/palettes.html)

In [50]:
from bokeh.palettes import viridis
mapper = LinearColorMapper(palette=viridis(10), low=0, high=1)

### Creating figure

In [51]:
TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom, tap"

In [52]:
p = figure(title="Confusion matrix MNIST",
           x_range=['0','1','2','3','4','5','6','7','8','9'], 
           y_range=['0','1','2','3','4','5','6','7','8','9'], 
           x_axis_location="above", plot_width=800, plot_height=400,
           tools=TOOLS, toolbar_location='below',
           tooltips=[('true', '@y_true'), ('pred', '@y_pred'), ('acc', '@acc')])

In [53]:
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 90

In [54]:
image_try = np.reshape(test_images[0], [test_images[0].shape[0], test_images[0].shape[1]])
image_try.shape

(28, 28)

In [57]:
p.rect(x="y_pred", y="y_true", width=1, height=1,
       source=source,
       fill_color={'field': 'acc', 'transform': mapper},
       line_color=None)

callback = CustomJS(args=dict(s1=source), code= """
        var a = cb_obj.indices;
        var data = s1.data;
          """)
source.selected.js_on_change('indices', callback)


In [58]:
color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="7pt",
                     ticker=FixedTicker(ticks=np.linspace(0,1,10)),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

In [59]:
show(p)
